## ACS Housing Units by Building Size for Urban Villages, City of Phoenix

- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

In [ ]:
import pandas as pd
import math
import numpy as np
import os

In [ ]:
import get_acs as get
import utilcalcs as calc
import geo_agg
from hou_size_vars import *

In [ ]:
bgp_10 = pd.read_csv('../data/geo/bgp_vil_10.csv')
bgp_20 = pd.read_csv('../data/geo/bgp_vil_20.csv')
for df in [bgp_10,bgp_20]: df.geoid = df.geoid.apply(lambda x: '{0:0>12}'.format(x))
    
#get ride of area & geo stuff not being useed
bgp_20 = bgp_20.drop(['aland20','awater20','lat20','lon20','land_acre'],axis=1)
bgp_10 = bgp_10.drop(['aland10','awater10','lat10','lon10','land_acre'],axis=1)

In [ ]:
#Search parameters - NOTE DIFFERENT VARIABLE # ASSIGNMENTS IN 2010 VS 2020
y1 = '2021'
y0 = '2013'

cols = 'group(B25024)'

source = 'acs/acs5'

#### Calculate building size columns function

In [ ]:
# function to sum select building columns & recalc MOES
def group_bsize(df,year):
    df['u_1_E'] = df.loc[:,U1E].sum(axis=1)
    df['u_1_M'] = df.apply(lambda x: (calc.get_moe(x[U1M])),axis=1)
    df['u_24_E'] = df.loc[:,U24E].sum(axis=1)
    df['u_24_M'] = df.apply(lambda x: (calc.get_moe(x[U24M])),axis=1)
    df['u_520_E'] = df.loc[:,U520E].sum(axis=1)
    df['u_520_M'] = df.apply(lambda x: (calc.get_moe(x[U520M])),axis=1)
    df['u_o20_E'] = df.loc[:,Uo20E].sum(axis=1)
    df['u_o20_M'] = df.apply(lambda x: (calc.get_moe(x[Uo20M])),axis=1)
    df['u_oth_E'] = df.loc[:,UOthE].sum(axis=1)
    df['u_oth_M'] = df.apply(lambda x: (calc.get_moe(x[UOthM])),axis=1)
    df = df.drop(columns=hou_vars).rename(hou_rename,axis=1)
    
    #add end-year into column name
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

In [ ]:
# make urban villages from Census Block Groups

def make_uvil(geo_df,year): 
    df = get.get_bgp(source,year,cols)
    df = df.filter(regex='(?<!A)$',axis=1) #drop non-estimate columns
    df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
    df = get.clean_data(df,df.columns)
    df = group_bsize(df,year)
    df = pd.merge(geo_df,df,how='left',left_on='geoid',right_on='GEO_ID')
    df = df.drop(['geoid','GEO_ID'],axis=1)
    df = geo_agg.sumgeo_cv(df,'name')
    return df

In [ ]:
# make Phoenix or U.S.
def make_table(year,geo):
    if geo == 'phoenix':
        df = get.get_phx(source,year,cols)
    elif geo == 'us':
        df = get.get_us(source,year,cols)
        df = df.drop(['NAME','us'],axis=1)
    else:
        pass
    df = df.filter(regex='(?<!A)$',axis=1) #drop non-estimate columns
    df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
    df = get.clean_data(df,df.columns)
    df = group_bsize(df,year)
    df = geo_agg.sumgeo_cv(df,'GEO_ID')
    return df

### Make a table with urban villages, city, and  U.S.

In [ ]:
dfY1 = make_uvil(bgp_20,y1)

In [ ]:
dfY1.head(3)

In [ ]:
phY1 = make_table(y1,'phoenix')
phY1.rename(columns={'GEO_ID':'name'},inplace=True)

In [ ]:
phY1

In [ ]:
usY1 = make_table(y1,'us')
usY1.rename(columns={'GEO_ID':'name'},inplace=True)

In [ ]:
usY1

## Build current year table for all geos & export to Excel

In [ ]:
final = pd.concat([dfY1,phY1,usY1])

In [ ]:
final

In [ ]:
final.to_excel(f'output/.xlsx',index=False)

In [ ]:
ct_y1.to_excel(f'../output/Housing/BuildSize_ACS_tract_{years[1]}.xlsx',index=False)